In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('/home/briggsc1-erau.edu/Downloads/housing.csv')

In [4]:
df.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

In [5]:
features = ['housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income']
target = ['median_house_value']

In [12]:
# split into 60-20-20 train-val-test
df = df.dropna()
x_train,x_test,y_train,y_test = train_test_split(df[features],df[target],
                                                test_size = 0.4, random_state = 0)
x_val,x_test,y_val,y_test = train_test_split(x_test,y_test,
                                                test_size = 0.5, random_state = 0)

In [13]:
# initialize and fit scaler and pca
scaler = StandardScaler()
pca = PCA()
scaler.fit(x_train)
pca.fit(scaler.transform(x_train))

PCA()

In [14]:
# transform the x_train and x_val sets to x_tr and x_va
x_tr = pca.transform(scaler.transform(x_train))
x_va = pca.transform(scaler.transform(x_val))

In [17]:
# investigate what PCA is doing to some specified rows
pca.transform(np.asarray((1,0,0,0,0,0)).reshape(1,6))

array([[-0.21758767, -0.39396996,  0.89184212, -0.02022467,  0.03942616,
         0.00987201]])

In [19]:
# PCA performs a linear transformation.
# PCA transformation can then be represented by matrix mult
print(pca.transform(np.eye(6))) # mult by this matrix performs PCA transform
# the columns are the principal components. but where do these columns come from?

[[-0.21758767 -0.39396996  0.89184212 -0.02022467  0.03942616  0.00987201]
 [ 0.48810057  0.09128683  0.11946419 -0.28254304  0.79724056 -0.15404471]
 [ 0.49383433 -0.12111585  0.06356704 -0.38319068 -0.29683843  0.70885096]
 [ 0.47235945 -0.1162693   0.0801676   0.85986928  0.041177    0.12593821]
 [ 0.4952944  -0.11392344  0.09599876 -0.17379094 -0.49660192 -0.6750351 ]
 [ 0.04583767  0.89178935  0.41311327  0.0578518  -0.16257613  0.04658616]]


In [26]:
# note the columns are unital.
pca_matrix = pca.transform(np.eye(6))
np.linalg.norm(pca_matrix[:,5])
# the columns are also mutually orthogonal
# take dot products of all pairs of distinct columns, get 0


1.0

In [28]:
# use numpy to find eigenvectors and eigenvalues
A = np.matrix(((1,6),(3,4)))
evals,evects = np.linalg.eig(A)

In [29]:
evals

array([-2.,  7.])

In [30]:
evects

matrix([[-0.89442719, -0.70710678],
        [ 0.4472136 , -0.70710678]])

In [40]:
# how about variance?
# warning: default ddof=0
np.var((1,2,3,4,5))
np.var((7,3,2))

4.666666666666667

In [41]:
# warning: default ddof=1
np.cov((7,3,2),(1,2,3))

array([[ 7. , -2.5],
       [-2.5,  1. ]])

In [48]:
# principal components are the eigenvectors of the covariance matrix of the data.
cm = pd.DataFrame(scaler.transform(x_train)).cov() # covariance matrix
evals, evecs = np.linalg.eig(cm)

In [49]:
evals

array([3.87393   , 1.06875416, 0.81799215, 0.15643487, 0.06880774,
       0.01457055])

In [52]:
print(pca_matrix)

[[-0.21758767 -0.39396996  0.89184212 -0.02022467  0.03942616  0.00987201]
 [ 0.48810057  0.09128683  0.11946419 -0.28254304  0.79724056 -0.15404471]
 [ 0.49383433 -0.12111585  0.06356704 -0.38319068 -0.29683843  0.70885096]
 [ 0.47235945 -0.1162693   0.0801676   0.85986928  0.041177    0.12593821]
 [ 0.4952944  -0.11392344  0.09599876 -0.17379094 -0.49660192 -0.6750351 ]
 [ 0.04583767  0.89178935  0.41311327  0.0578518  -0.16257613  0.04658616]]


In [51]:
print(evecs)

[[ 0.21758767 -0.39396996  0.89184212 -0.02022467 -0.03942616  0.00987201]
 [-0.48810057  0.09128683  0.11946419 -0.28254304 -0.79724056 -0.15404471]
 [-0.49383433 -0.12111585  0.06356704 -0.38319068  0.29683843  0.70885096]
 [-0.47235945 -0.1162693   0.0801676   0.85986928 -0.041177    0.12593821]
 [-0.4952944  -0.11392344  0.09599876 -0.17379094  0.49660192 -0.6750351 ]
 [-0.04583767  0.89178935  0.41311327  0.0578518   0.16257613  0.04658616]]


In [56]:
# what are the representations of the principal components in housing age?
np.matrix((1,0,0,0,0,0)).reshape(1,6)*evecs
np.matrix((0,0,0,0,0,1)).reshape(1,6)*evecs

matrix([[-0.04583767,  0.89178935,  0.41311327,  0.0578518 ,  0.16257613,
          0.04658616]])